# 1. record point cloud

In [1]:
import sys
import os

# Get the current working directory (used to replace __file__)
current_dir = os.getcwd()

target_path1 = os.path.abspath(os.path.join(current_dir, '../3_move'))
target_path2 = os.path.abspath(os.path.join(current_dir, '../1_getPointCloud'))

# Add the paths to sys.path
if target_path1 not in sys.path:
    sys.path.append(target_path1)
    
if target_path2 not in sys.path:
    sys.path.append(target_path2)
    
# Check if the paths were added successfully
print("Current Python Path:")
print("\n".join(sys.path))

from ImageRecognizer import ImageRecognizer
image_recognizer = ImageRecognizer(top_dir="/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/cubes/")

from utils import matrix_to_rpy_and_translation

from PickAndPlace import PickAndPlace
pick_place = PickAndPlace(approach_distance=0.2)

Current Python Path:

/opt/ros_ws/devel/lib/python3/dist-packages
/opt/ros/noetic/lib/python3/dist-packages
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/3_move
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/1_getPointCloud


INFO - 2025-02-27 10:15:06,621 - topics - topicmanager initialized


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[INFO] [1740651306.876221, 0.000000]: Waiting for move_group action server...
[INFO] [1740651307.931848, 0.000000]: Table added to the scene to prevent collision below z = 0.001


[ WARN] [1740651306.903528877]: Link zed2_holder has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


[INFO] [1740651307.936575, 0.000000]: Wall 'wall_right' added to the scene with rotation theta=-0.6283185307179586 radians at position: 0.0, 0.8, 0.0
[INFO] [1740651307.940274, 0.000000]: Wall 'wall_left' added to the scene with rotation theta=0.6283185307179586 radians at position: 0.0, -0.8, 0.0
[INFO] [1740651307.941920, 16.244000]: MoveRobot initialized successfully.
[INFO] [1740651308.179927, 16.282000]: Current joint values: [-0.0001266525945400332, -0.784100866996857, 2.3842241139604425e-05, -2.3586074891815416, 5.028194698830646e-06, 1.5713374593636535, 0.7853932313514509]
[INFO] [1740651308.200804, 16.502000]: Waiting for gripper action servers...
[INFO] [1740651308.464855, 16.760000]: Gripper action servers ready.


In [2]:
from save_point_cloud import PointCloudSaver
import open3d as o3d
import rospy
point_cloud_saver = PointCloudSaver()

# Wait for data to be ready
rospy.loginfo("Waiting for data...")
rospy.sleep(1)  # Wait for topic data to be published

# Save the point cloud
world_file = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud_world3.ply"
point_cloud_saver.save_point_clouds(world_file)

[INFO] [1740651308.567916, 16.847000]: Waiting for data...
[INFO] [1740651308.703351, 16.987000]: Received image message.
[INFO] [1740651308.746247, 16.991000]: Received point cloud data.


[ WARN] [1740651308.641501423, 16.922000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


[INFO] [1740651311.018410, 19.178000]: Color range - Min: [0.10196078 0.10196078 0.10196078], Max: [0.70196078 0.60784314 0.60784314]
[INFO] [1740651311.019503, 19.178000]: Requesting transform from world to left_camera_link_optical...
[INFO] [1740651311.020483, 19.178000]: Transform found: header: 
  seq: 0
  stamp: 
    secs: 19
    nsecs: 167000000
  frame_id: "world"
child_frame_id: "left_camera_link_optical"
transform: 
  translation: 
    x: 0.2097305007671303
    y: -0.0600189768488691
    z: 0.5616907560719664
  rotation: 
    x: 0.6588839359701208
    y: 0.658817877765095
    z: 0.2567227585473902
    w: 0.2567574499433966
[INFO] [1740651311.295351, 19.463000]: Transformed point cloud saved to /opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud_world3.ply


In [3]:
import open3d as o3d
import numpy as np
from utils import filter_point_cloud_by_depth_and_range, filter_point_cloud_by_depth

# zed_ply_path = "mesh/zed_point_cloud_world3.ply"
zed_ply_path = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud_world3.ply"

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # Size of the coordinate axes, can be adjusted as needed
    origin=[0, 0, 0]  # Origin of the coordinate axes
)

# Read the point cloud file
point_cloud = o3d.io.read_point_cloud(zed_ply_path)
if not point_cloud.has_points():
    raise ValueError(f"Failed to read point cloud from {zed_ply_path}")
filtered_point_cloud = filter_point_cloud_by_depth_and_range(point_cloud, depth_threshold=0.02, range=[0.001, -0.8, 1, 1.6])
o3d.visualization.draw_geometries([filtered_point_cloud, coordinate_frame], window_name="Filtered Point Cloud")

In [4]:
# from utils import calculate_max_layer

# # Example usage
# max_layer = calculate_max_layer(filtered_point_cloud, layer_height=0.04)
# print(f"MaxLayer: {max_layer}")

# 2. Perception

## 2.1 Coarse and Fine registration 

In [5]:
import copy
import open3d as o3d
import numpy as np

def register_and_filter(pointcloud, mesh, voxel_size=0.01):
    # Convert mesh to point cloud
    if isinstance(mesh, o3d.geometry.TriangleMesh):
        mesh_pointcloud = mesh.sample_points_uniformly(number_of_points=1000)
    elif isinstance(mesh, o3d.geometry.PointCloud):
        mesh_pointcloud = copy.deepcopy(mesh)
    
    # Downsample the point cloud and compute features
    def preprocess_point_cloud(pcd, voxel_size):
        pcd_down = pcd.voxel_down_sample(voxel_size)
        pcd_down.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*2, 
                max_nn=30
            )
        )
        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            pcd_down,
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*5, 
                max_nn=100
            )
        )
        return pcd_down, pcd_fpfh

    # Coarse registration
    def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
        distance_threshold = voxel_size * 1.5
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, True,
            distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            4,
            [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
        )
        return result

    # Fine registration
    def refine_registration(source, target, initial_transformation, voxel_size):
        distance_threshold = voxel_size * 1  # Reduce the threshold for higher accuracy
        result = o3d.pipelines.registration.registration_icp(
            source, target, distance_threshold, initial_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000000)
        )
        return result

    # Execute point cloud preprocessing
    source_down, source_fpfh = preprocess_point_cloud(mesh_pointcloud, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(pointcloud, voxel_size)

    # Execute registration
    coarse_result = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    refined_result = refine_registration(mesh_pointcloud, pointcloud, coarse_result.transformation, voxel_size)

    # Transform the mesh point cloud
    transform = refined_result.transformation
    transformed_mesh_pointcloud = mesh_pointcloud.transform(transform)

    # Create bounding box
    oriented_bounding_box = transformed_mesh_pointcloud.get_oriented_bounding_box()
    center = oriented_bounding_box.center
    extent = oriented_bounding_box.extent
    rotation_matrix = oriented_bounding_box.R

    # Expand the bounding box
    margin = voxel_size
    expanded_extent = extent + 0.5 * margin
    expanded_bounding_box = o3d.geometry.OrientedBoundingBox(
        center=center,
        extent=expanded_extent,
        R=rotation_matrix
    )

    # Filter the point cloud
    indices_inside_box = expanded_bounding_box.get_point_indices_within_bounding_box(pointcloud.points)
    indices_outside_box = list(set(range(len(pointcloud.points))) - set(indices_inside_box))

    # Separate the point cloud
    remaining_pointcloud = pointcloud.select_by_index(indices_outside_box)
    deleted_pointcloud = pointcloud.select_by_index(indices_inside_box)

    return transform, remaining_pointcloud, deleted_pointcloud, refined_result.fitness


In [6]:
# Read the cube
# Define file paths
cube_obj_path = "mesh/cube_0.obj"
zed_ply_path = "mesh/zed_point_cloud_world3.ply"

# Read the cube_0.obj mesh
cube_mesh = o3d.io.read_triangle_mesh(cube_obj_path)
cube_mesh.compute_vertex_normals()  # Compute normals for better visualization
# cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)  # Convert to point cloud
# cube_point_cloud = cube_mesh.sample_points_poisson_disk(number_of_points=1000)

# Remove the lower part of the cube to prevent flipping along the z-axis
# cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.015)
# o3d.visualization.draw_geometries([cube_point_cloud])


coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # Size of the coordinate axes, can be adjusted as needed
    origin=[0, 0, 0]  # Origin of the coordinate axes
)


In [7]:
def check_transform_z_axis_alignment(transform, tolerance=0.1):
    """
    Check if the Z-axis of the transform is parallel to the Z-axis of the world coordinate system.
    Allows a certain tolerance range to check if it is parallel or anti-parallel.
    
    Args:
    - transform: 4x4 transformation matrix
    - tolerance: Tolerance range for checking, default is 0.1
    
    Returns:
    - True: If the Z-axis is parallel or anti-parallel
    - False: If the Z-axis is not parallel
    - The corrected transform
    """
    z_axis = np.array([0, 0, 1])  # Z-axis of the world coordinate system
    transform_z_axis = transform[:3, 2]  # Get the Z-axis of the transform (i.e., the third column of the rotation matrix)

    # Compute the angle between the transform's Z-axis and the world coordinate system's Z-axis
    dot_product = np.dot(transform_z_axis, z_axis)
    # Compute the cosine of the angle, if close to 1 or -1, it means parallel or anti-parallel
    if np.abs(dot_product) > (1 - tolerance):
        return True
    return False

def align_transform_z_axis(transform):
    """
    If the Z-axis of the transform is anti-parallel to the Z-axis of the world coordinate system,
    rotate by 180 degrees around the X-axis (np.pi) to flip the direction of the Z-axis.
    
    Args:
    - transform: 4x4 transformation matrix
    
    Returns:
    - The corrected transform matrix
    """
    z_axis_world = np.array([0, 0, 1])  # Z-axis of the world coordinate system
    transform_z_axis = transform[:3, 2]  # Get the Z-axis of the transform (the third column of the rotation matrix)

    # Check if the Z-axis is anti-parallel to the world coordinate system's Z-axis
    if np.dot(transform_z_axis, z_axis_world) < 0:  # Z-axis is anti-parallel
        print("Correcting Z axis")
        # Create a rotation matrix to rotate 180 degrees around the X-axis
        rotation_matrix = np.eye(4)
        rotation_matrix[1, 1] = -1  # Rotate the matrix by 180 degrees around the X-axis
        rotation_matrix[2, 2] = -1  # Rotate the matrix by 180 degrees around the X-axis
        
        # Perform matrix multiplication, applying the rotation matrix to the original transform
        transform = np.dot(rotation_matrix, transform)

    return transform


In [8]:
import rospy
import tf
import numpy as np
from geometry_msgs.msg import TransformStamped

class TransformBroadcaster:
    def __init__(self):
        # Try to initialize the ROS node, avoid initializing multiple times
        try:
            rospy.init_node('tf_broadcaster_node')
        except rospy.exceptions.ROSException:
            pass  # If the node is already initialized, do nothing

        # Create a TransformBroadcaster instance
        self.br = tf.TransformBroadcaster()

        # Assume T is the given 4x4 transformation matrix
        self.T = np.ones((4, 4))  # Set to a 4x4 matrix

        # Set a timer to call the broadcast_transform function every 100 milliseconds
        self.timer = rospy.Timer(rospy.Duration(0.1), self.broadcast_transform)

        # Store the timestamp of the last sent transformation
        self.last_sent_time = None

    def broadcast_transform(self, event):
        try:
            # Extract the translation and rotation parts from the 4x4 matrix
            translation = self.T[0:3, 3]  # Translation part (x, y, z)
            rotation_matrix = self.T[0:3, 0:3]  # Rotation matrix part

            # Create a complete 4x4 matrix, including rotation and homogeneous coordinates
            full_matrix = np.eye(4)
            full_matrix[0:3, 0:3] = rotation_matrix
            full_matrix[0:3, 3] = translation

            # Create a quaternion to represent the rotation
            quaternion = tf.transformations.quaternion_from_matrix(full_matrix)

            # Get the current timestamp
            current_time = rospy.Time.now()

            # Check if the last sent timestamp and the current timestamp are the same
            if self.last_sent_time is None or current_time != self.last_sent_time:
                # Publish the transformation
                self.br.sendTransform(
                    (translation[0], translation[1], translation[2]),  # Translation part
                    (quaternion[0], quaternion[1], quaternion[2], quaternion[3]),  # Rotation part (quaternion)
                    current_time,  # Use the current timestamp
                    "cube",  # Child frame name
                    "world"   # Parent frame name
                )
                # Update the last sent timestamp
                self.last_sent_time = current_time
        except:
            pass

    def update(self, T):
        self.T = T
        
    def stop(self):
        # Stop the timer
        self.timer.shutdown()


## 2.2 Grasp Generation

In [9]:
from utils import *  # Assuming the create_grasp_mesh function is in utils.py
import numpy as np
import open3d as o3d

def generate_gripper_from_transform(T: np.ndarray):
    """
    Generates a robotic gripper mesh from a given 4x4 transformation matrix,
    with additional rotations around x and y axes.

    Args:
        T: 4x4 transformation matrix (numpy array).
        
    Returns:
        gripper_meshes: List of meshes representing the gripper.
    """
    # Extract the rotation matrix (3x3)
    rotation_matrix = T[:3, :3]

    # Extract the translation vector
    translation = T[:3, 3]

    # Set the gripper's center point position, usually the translation vector
    center_point = translation

    # Create a rotation matrix for -90 degrees around the x-axis
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(-np.pi/2), -np.sin(-np.pi/2)],
        [0, np.sin(-np.pi/2), np.cos(-np.pi/2)]
    ])

    # Create a rotation matrix for 90 degrees around the y-axis
    R_y = np.array([
        [np.cos(np.pi/2), 0, np.sin(np.pi/2)],
        [0, 1, 0],
        [-np.sin(np.pi/2), 0, np.cos(np.pi/2)]
    ])
    
    R_z = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    # Combine rotation matrices, first rotate around the x-axis, then around the y-axis
    combined_rotation = R_z @ rotation_matrix @ R_x 

    # Call create_grasp_mesh function to generate the gripper
    gripper_meshes = create_grasp_mesh(
        center_point=center_point, 
        rotation_matrix=combined_rotation,
        width=0.25
    )
    # Call create_grasp_mesh function to generate the gripper with a different rotation
    gripper_meshes_rotate = create_grasp_mesh(
        center_point=center_point, 
        rotation_matrix=rotation_matrix @ R_x,
        width=0.25
    )

    return gripper_meshes, gripper_meshes_rotate


In [10]:
# Test code: Pass in a 4x4 transformation matrix
T = np.array([
    [1, 0, 0, 0.1],  # Rotation matrix and translation
    [0, 1, 0, 0.2],
    [0, 0, 1, 0.3],
    [0, 0, 0, 1]
])

# Call the function to generate the gripper
gripper_meshes, _ = generate_gripper_from_transform(T)

# Visualize the generated gripper
o3d.visualization.draw_geometries(gripper_meshes)


In [11]:
def check_grasp_collision(
    grasp_meshes: Sequence[o3d.geometry.TriangleMesh],
    object_pcd: o3d.geometry.TriangleMesh,
    num_colisions: int = 10,
    tolerance: float = 0.00001
) -> bool:
    """
    Checks for collisions between a gripper grasp pose and target object
    using point cloud sampling.

    Args:
        grasp_meshes: List of mesh geometries representing the gripper components
        object_mesh: Triangle mesh of the target object
        num_collisions: Threshold on how many points to check
        tolerance: Distance threshold for considering a collision (in meters)

    Returns:
        bool: True if collision detected between gripper and object, False otherwise
    """
    # Combine gripper meshes
    combined_gripper = o3d.geometry.TriangleMesh()
    for mesh in grasp_meshes:
        combined_gripper += mesh  # Combine multiple gripper meshes

    # Sample points from both meshes
    num_points = 5000  # Sample 5000 points from both gripper and target object
    #######################TODO#######################
    # Uniformly sample point clouds from both the gripper and object meshes
    gripper_pcd = combined_gripper.sample_points_uniformly(number_of_points=num_points)
    gripper_points = np.asarray(gripper_pcd.points)  # Point coordinates of the gripper point cloud
    object_points = np.asarray(object_pcd.points)  # Point coordinates of the target object point cloud
    ##################################################
    
    # Build KDTree for object points
    is_collision = False
    #######################TODO#######################
    collision_count = 0
    # Build a KDTree for the target object point cloud
    object_kdtree = o3d.geometry.KDTreeFlann(object_pcd)
    for gripper_point in gripper_points:
        # For each gripper point, find the nearest point in the target object point cloud
        _, _, distances = object_kdtree.search_knn_vector_3d(gripper_point, 1)  # Find the nearest neighbor
        
        # If the distance to the nearest neighbor is less than the tolerance, consider it a collision
        if distances[0] <= tolerance:
            collision_count += 1
            
            # Exit early if enough collisions are detected
            if collision_count >= num_colisions:
                is_collision = True
                break
    #######################TODO#######################

    return is_collision


## 2.3 identify image of cubes

In [12]:
# from openai import OpenAI
# from io import BytesIO
# from PIL import Image
# import base64
# import json
# SYSTEM_PROMPT = """Please act as an image recognition agent. 
# You will be given a square face of a block, 
# which is projected from a point cloud. 
# Your task is to recognize the following:

# Determine if this is a block face.
# Each face contains only one letter, 
# one pattern (just detect whether it's a pattern, no need to identify the exact pattern), 
# or is blank (only wood texture). 
# Please detect whether it is a letter, 
# a pattern, or blank. 
# Each of these may be rotated. 
# Please analyze all possible rotations in a clockwise direction: 0°, 90°, 180°, and 270°.
# There might be a circular border around the face. 
# Please detect if this border exists. 
# It's confirmed that the color of the border matches the color of the letter or pattern.
# The expected output is a JSON in the following format:
# {
#     "check": true/false, 
#     "c": char/"pattern"/"blank", 
#     "color": "green"/"yellow"/"red"/"blue"/"None", 
#     "rotation": 0/90/180/270, 
#     "circle": true/false
# }
# """

# api_key=""
# client = OpenAI(api_key=api_key)

# def encode_image(image, quality=100):
#     if image.mode != 'RGB':
#         image = image.convert('RGB')  # Convert to RGB
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG", quality=quality) 
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")

# def gpt4o_analysis(image_path, quality=50):
#     with Image.open(image_path) as img:
#         img_b64_str = encode_image(img, quality=quality)
#     img_type = "image/jpeg"
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": SYSTEM_PROMPT},
#                     {
#                         "type": "image_url",
#                         "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"},
#                     },
#                 ],
#             }from openai import OpenAI
# from io import BytesIO
# from PIL import Image
# import base64
# import json
# SYSTEM_PROMPT = """Please act as an image recognition agent. 
# You will be given a square face of a block, 
# which is projected from a point cloud. 
# Your task is to recognize the following:

# Determine if this is a block face.
# Each face contains only one letter, 
# one pattern (just detect whether it's a pattern, no need to identify the exact pattern), 
# or is blank (only wood texture). 
# Please detect whether it is a letter, 
# a pattern, or blank. 
# Each of these may be rotated. 
# Please analyze all possible rotations in a clockwise direction: 0°, 90°, 180°, and 270°.
# There might be a circular border around the face. 
# Please detect if this border exists. 
# It's confirmed that the color of the border matches the color of the letter or pattern.
# The expected output is a JSON in the following format:
# {
#     "check": true/false, 
#     "c": char/"pattern"/"blank", 
#     "color": "green"/"yellow"/"red"/"blue"/"None", 
#     "rotation": 0/90/180/270, 
#     "circle": true/false
# }
# """

# api_key=""
# client = OpenAI(api_key=api_key)

# def encode_image(image, quality=100):
#     if image.mode != 'RGB':
#         image = image.convert('RGB')  # Convert to RGB
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG", quality=quality) 
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")

# def gpt4o_analysis(image_path, quality=50):
#     with Image.open(image_path) as img:
#         img_b64_str = encode_image(img, quality=quality)
#     img_type = "image/jpeg"
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": SYSTEM_PROMPT},
#                     {
#                         "type": "image_url",
#                         "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"},
#                     },
#                 ],
#             }
#         ],
#     )
#     return response.choices[0].message.content
#         ],
#     )
#     return response.choices[0].message.content

In [13]:
import logging
logging.basicConfig(level=logging.ERROR)
import matplotlib.pyplot as plt
import cv2
from PointCloud2Image import enlarge_points_as_cubes, max_downsample_image, pointcloud_to_top_view_image_color, interpolate_sparse_image, pointcloud_to_colored_image_with_filling, triangle_mesh_to_image

try:
    broadcaster
except NameError:
    broadcaster = TransformBroadcaster()           
    
cube_num = 4
def pointcloud_process(point_cloud, slice_tolerance=0.005):
    '''
        Identify the cubes
        Returns:
            [
                json,
                T
            ]
    '''
    orignal_point_cloud = copy.deepcopy(point_cloud)
    T = []
    remaining_pointcloud_count = 10000
    countdown = 50
    # Use open3d to visualize the point cloud
    # o3d.visualization.draw_geometries([layer_point_cloud], window_name=f"Layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
    movecount = 0
    while remaining_pointcloud_count > 50 and countdown > 0:
        cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
        cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
        transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(point_cloud, cube_point_cloud)
        remaining_pointcloud_count = len(remaining_pointcloud.points)
        # return transform, remaining_pointcloud, deleted_pointcloud
        if fitness > 0.01:
            countdown = countdown - 1
        print(fitness)
        if check_transform_z_axis_alignment(transform) and fitness > 0.50 and np.array_equal(transform, align_transform_z_axis(transform)):
        # if fitness > 0.70:
            if movecount >= cube_num-1:
                break
            print(movecount)
            

            broadcaster.update(transform)
            cube_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                size=0.1,  # Coordinate axis size, can be adjusted as needed
                origin=[0, 0, 0]  # Origin of the coordinate axis
            )
            theta = np.radians(45)  # Convert angle to radians
            transform_matrix_x_180 = np.array([
                [1, 0, 0, 0],
                [0, -1, 0, 0],
                [0, 0, -1, 0],
                [0, 0, 0, 1]
            ])
            transform_matrix_z_90 = np.array([
                [0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]
            ])
            graps_transform = transform @ transform_matrix_x_180
            graps_transform_rotate = transform @ transform_matrix_x_180 @ transform_matrix_z_90
                        # o3d.visualization.draw_geometries([coordinate_frame, remaining_pointcloud], window_name="remaining_pointcloud")
            cube_point_cloud_transormed = copy.deepcopy(deleted_pointcloud)
            cube_point_cloud_transormed = cube_point_cloud_transormed.transform(np.linalg.inv(graps_transform))
            cube_point_cloud_transormed_cubes = enlarge_points_as_cubes(cube_point_cloud_transormed)

            cube_top_image = triangle_mesh_to_image(cube_point_cloud_transormed_cubes, image_size=(100, 100))
            cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
            grasp_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
                size=0.1,  # Coordinate axis size, can be adjusted as needed
                origin=[0, 0, 0]  # Origin of the coordinate axis
            )
            grasp_mesh, gripper_meshes_rotate = generate_gripper_from_transform(graps_transform)
            # Apply transformation matrix to the coordinate frame
            grasp_coordinate_frame.transform(graps_transform)
            grasp_final_matrix = None
            # o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[orignal_point_cloud, coordinate_frame], window_name="remaining_pointcloud")
            if check_grasp_collision(grasp_mesh, orignal_point_cloud):
            # if check_grasp_collision(grasp_mesh, deleted_pointcloud):
                # If collision
                grasp_mesh = []
            else:
                grasp_final_matrix = graps_transform
            if check_grasp_collision(gripper_meshes_rotate, orignal_point_cloud):
            # if check_grasp_collision(gripper_meshes_rotate, deleted_pointcloud):
                gripper_meshes_rotate = []
            else:
                grasp_final_matrix = graps_transform_rotate
            o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[grasp_coordinate_frame, deleted_pointcloud, coordinate_frame], window_name="deleted_pointcloud")
            # o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[grasp_coordinate_frame, remaining_pointcloud, coordinate_frame], window_name="deleted_pointcloud")
            if grasp_final_matrix is not None: # Can move
                countdown = 50
                print(grasp_final_matrix)
                movecount += 1
                pick_rpy, pick_pos = matrix_to_rpy_and_translation(grasp_final_matrix)
                pick_pos_ = [a + b for a, b in zip(pick_pos, [0, 0, 0.10])]
                pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
                # pick_place.move(pick_pos_, pick_rpy)
                print(pick_pos_, pick_rpy)
                # cube_top_image = point_cloud_to_image(cube_point_cloud_transormed)
                plt.imsave(f"test_{movecount}.png", cube_top_image)
                plt.show()
                point_cloud = remaining_pointcloud
                # TODO: Identify the first face
            
                T.append(grasp_final_matrix)
        else:
            cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)
            cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
    print(T)
    return T
            
Ts = pointcloud_process(filtered_point_cloud)
broadcaster.stop()


[ WARN] [1740651314.662631273, 22.702000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651314.837094669, 22.875000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.017626345, 23.044000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.153027244, 23.176000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.405724564, 23.414000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.588643367, 23.589000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


0.7132001736864959
Correcting Z axis


[ WARN] [1740651315.719151111, 23.712000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.813700350, 23.807000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.882705297, 23.872000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651315.955085923, 23.946000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651316.028133980, 24.017000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651316.092607616, 24.080000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651316.169548950, 24.141000000]: New joint state for joint 'pan

0.8138704267345798
0
[[-0.99907385 -0.04283481  0.00407787  0.69954367]
 [-0.04284633  0.9990778  -0.00278183 -0.49816625]
 [-0.00395495 -0.00295397 -0.99998782  0.02222376]
 [ 0.          0.          0.          1.        ]]
[0.6995436660410469, -0.4981662463785952, 0.1222237627452182] [-3.1386386530494828, 0.003954958007977849, -3.0987328681412816]


[ WARN] [1740651319.073015722, 26.868000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651319.517254900, 27.282000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651319.748040983, 27.477000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


...

0.6438142723589904


[ WARN] [1740651328.773793262, 35.980000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651329.063637901, 36.198000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651329.064918102, 36.251000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651329.264429998, 36.446000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651329.445935078, 36.618000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


0.8396200055881531
2
[[ 9.98528567e-01  5.41574008e-02  2.77071235e-03  6.99354077e-01]
 [ 5.41583409e-02 -9.98532325e-01 -2.65339020e-04 -2.98875994e-01]
 [ 2.75227577e-03  4.15005775e-04 -9.99996126e-01  2.22654666e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[0.6993540772779313, -0.29887599422475136, 0.12226546656411075] [3.141177646230598, -0.0027522792443199773, 0.054185056894009084]


[ WARN] [1740651331.196977142, 38.308000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651331.325425337, 38.422000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651331.427655081, 38.523000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651331.684640387, 38.777000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


0.8040225960643119
[array([[-0.99907385, -0.04283481,  0.00407787,  0.69954367],
       [-0.04284633,  0.9990778 , -0.00278183, -0.49816625],
       [-0.00395495, -0.00295397, -0.99998782,  0.02222376],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), array([[ 0.99885905,  0.04762965,  0.00346571,  0.69942123],
       [ 0.04765514, -0.99883427, -0.00768739, -0.39871937],
       [ 0.00309552,  0.00784378, -0.99996445,  0.02229673],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), array([[ 9.98528567e-01,  5.41574008e-02,  2.77071235e-03,
         6.99354077e-01],
       [ 5.41583409e-02, -9.98532325e-01, -2.65339020e-04,
        -2.98875994e-01],
       [ 2.75227577e-03,  4.15005775e-04, -9.99996126e-01,
         2.22654666e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])]


In [14]:
Ts.__len__()

3

In [15]:
import numpy as np
import open3d as o3d
import copy

def generate_pascal_triangle_transforms(t, T):
    """
    Generate a list of transformation matrices for a Pascal triangle arrangement.
    Each cube's center position is used as the translation part of the transform matrix.
    """
    level = 1
    total = 1
    while total < t:
        level += 1
        total += level

    transforms = []
    cube_size = 0.045  # 4.5 cm
    spacing_xy = cube_size * 1.25  # Increase spacing to 1.25 times the cube size
    spacing_z = cube_size * 1.05   # Use the same spacing in the vertical direction

    current_pos = 0
    for row in range(level-1, -1, -1):
        for col in range(row + 1):
            if current_pos >= t:
                break

            center_x = 0
            center_y = (col - row/2) * spacing_xy
            center_z = (level - 1 - row) * spacing_z

            # Create local transformation matrix
            local_transform = np.eye(4)
            local_transform[:3, 3] = [center_x, center_y, center_z]

            # Combine local transformation with T transformation
            transform = np.dot(T, local_transform)
            transforms.append(transform)
            current_pos += 1

        if current_pos >= t:
            break

    return transforms

def create_coordinate_frame(size=0.1, transform=None):
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    if transform is not None:
        frame.transform(transform)
    return frame

def visualize_pascal_triangle(transforms, T):
    # Create a cube centered at the origin
    cube = o3d.geometry.TriangleMesh.create_box(
        width=0.045,
        height=0.045, 
        depth=0.045
    )
    # Move the cube to be centered at the origin
    cube.translate([-0.045/2, -0.045/2, -0.045/2])
    cube.compute_vertex_normals()

    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Add world coordinate frame
    world_frame = create_coordinate_frame(size=0.2)
    vis.add_geometry(world_frame)

    # Add T coordinate frame
    t_frame = create_coordinate_frame(size=0.2, transform=T)
    vis.add_geometry(t_frame)

    # Add all cubes and their local coordinate frames
    for transform in transforms:
        # Add cube
        cube_copy = copy.deepcopy(cube)
        cube_copy.transform(transform)
        vis.add_geometry(cube_copy)
        
        # Add local coordinate frame
        local_frame = create_coordinate_frame(size=0.05, transform=transform)
        vis.add_geometry(local_frame)

    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])

    ctr = vis.get_view_control()
    ctr.set_zoom(0.2)  # Adjust zoom to fit larger spacing
    ctr.set_front([-0.8, -0.5, 0.5])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 0, 1])

    vis.run()
    vis.destroy_window()


In [16]:
import numpy as np
import open3d as o3d
import copy

def generate_cubes_stack(t, T):
    """
    Generate a list of transformation matrices for a stack of cubes.
    Each cube is translated along the Z-axis, creating a tower-like structure.
    """
    transforms = []
    cube_size = 0.045  # 4.5 cm
    spacing_z = cube_size * 1.05   # Use the same spacing in the vertical direction

    for i in range(t):
        # Calculate the center position of the cube
        center_x = 0
        center_y = 0
        center_z = i * spacing_z  # Stack the cubes along the Z-axis

        # Create local transformation matrix
        local_transform = np.eye(4)
        local_transform[:3, 3] = [center_x, center_y, center_z]

        # Combine local transformation with T transformation
        transform = np.dot(T, local_transform)
        transforms.append(transform)

    return transforms

def create_coordinate_frame(size=0.1, transform=None):
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    if transform is not None:
        frame.transform(transform)
    return frame

def visualize_cubes_stack(transforms, T):
    # Create a cube centered at the origin
    cube = o3d.geometry.TriangleMesh.create_box(
        width=0.045,
        height=0.045, 
        depth=0.045
    )
    # Move the cube to be centered at the origin
    cube.translate([-0.045/2, -0.045/2, -0.045/2])
    cube.compute_vertex_normals()

    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Add world coordinate frame
    world_frame = create_coordinate_frame(size=0.2)
    vis.add_geometry(world_frame)

    # Add T coordinate frame
    t_frame = create_coordinate_frame(size=0.2, transform=T)
    vis.add_geometry(t_frame)

    # Add all cubes and their local coordinate frames
    for transform in transforms:
        # Add cube
        cube_copy = copy.deepcopy(cube)
        cube_copy.transform(transform)
        vis.add_geometry(cube_copy)
        
        # Add local coordinate frame
        local_frame = create_coordinate_frame(size=0.05, transform=transform)
        vis.add_geometry(local_frame)

    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])

    ctr = vis.get_view_control()
    ctr.set_zoom(0.2)  # Adjust zoom to fit larger spacing
    ctr.set_front([-0.8, -0.5, 0.5])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 0, 1])

    vis.run()
    vis.destroy_window()


# 3. Motion Planning, movement, and grasp

In [17]:
from utils import generate_pascal_triangle_transforms
import numpy as np

In [26]:
# t = Ts.__len__()
t = 3
# 创建T矩阵（示例：绕Z轴旋转45度并平移）
T = np.eye(4)
theta = 0
T[:3, :3] = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])
T[:3, 3] = [0.3, 0.00, 0]

aim_transforms = generate_pascal_triangle_transforms(t, T)
# 生成组成塔的3个cubes的位置
aim_transforms = generate_cubes_stack(t, T)

# visualize_pascal_triangle(aim_transforms, T)
visualize_cubes_stack(aim_transforms, T)

[ WARN] [1740651614.674563608, 312.606000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651616.726453533, 314.541000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651616.864939457, 314.678000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651622.253642263, 319.869000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


In [30]:
aim_transforms

[array([[1. , 0. , 0. , 0.3],
        [0. , 1. , 0. , 0. ],
        [0. , 0. , 1. , 0. ],
        [0. , 0. , 0. , 1. ]]),
 array([[1.     , 0.     , 0.     , 0.3    ],
        [0.     , 1.     , 0.     , 0.     ],
        [0.     , 0.     , 1.     , 0.04725],
        [0.     , 0.     , 0.     , 1.     ]]),
 array([[1.    , 0.    , 0.    , 0.3   ],
        [0.    , 1.    , 0.    , 0.    ],
        [0.    , 0.    , 1.    , 0.0945],
        [0.    , 0.    , 0.    , 1.    ]])]

[ WARN] [1740651713.372077761, 409.104000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651714.530061389, 410.237000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651733.929986064, 429.311000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


In [20]:
Ts

[array([[-0.99907385, -0.04283481,  0.00407787,  0.69954367],
        [-0.04284633,  0.9990778 , -0.00278183, -0.49816625],
        [-0.00395495, -0.00295397, -0.99998782,  0.02222376],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.99885905,  0.04762965,  0.00346571,  0.69942123],
        [ 0.04765514, -0.99883427, -0.00768739, -0.39871937],
        [ 0.00309552,  0.00784378, -0.99996445,  0.02229673],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 9.98528567e-01,  5.41574008e-02,  2.77071235e-03,
          6.99354077e-01],
        [ 5.41583409e-02, -9.98532325e-01, -2.65339020e-04,
         -2.98875994e-01],
        [ 2.75227577e-03,  4.15005775e-04, -9.99996126e-01,
          2.22654666e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])]

In [21]:
# # debug:
# import numpy as np

# Ts = [np.array([[-0.30936624, -0.95090032,  0.00900657,  0.46123814],
#         [-0.95038183,  0.30949514,  0.03141855, -0.07133785],
#         [-0.0326634 ,  0.00116016, -0.99946574,  0.03173297],
#         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
#  np.array([[ 0.9979552 , -0.0391174 , -0.05054952,  0.61442553],
#         [-0.04106496, -0.99843052, -0.03808115,  0.10012986],
#         [-0.04898055,  0.0400791 , -0.99799528,  0.01518867],
#         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
#  # np.array([[ 0.97671909, -0.17776618,  0.12007919,  0.4728227 ],
#  #        [-0.19286849, -0.97274425,  0.12872592,  0.08761278],
#  #        [ 0.09392323, -0.14888855, -0.98438337,  0.02699751],
#  #        [ 0.        ,  0.        ,  0.        ,  1.        ]])
#      ]


In [35]:
# demostrate the movement to the head of identified cubes
count = 0
for T in Ts:
    count += 1
    pick_rpy, pick_pos = matrix_to_rpy_and_translation(T)
    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.04, 0.00, 0.10])]
    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
    print(count, pick_rpy, pick_pos)
    pick_place.move(pick_pos_, pick_rpy)

[ WARN] [1740651951.381493126, 641.043000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651951.442287528, 641.102000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


1 [-3.1386386530494828, 0.003954958007977849, -3.0987328681412816] [0.6995436660410469, -0.4981662463785952, 0.022223762745218197]


[ WARN] [1740651951.717232999, 641.372000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651951.941114848, 641.577000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651952.286823303, 641.904000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651952.435731438, 642.036000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651952.583508576, 642.172000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651952.650339458, 642.238000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651952.720482595, 642.307000000]: New joint state for joi

2 [3.133748760192116, -0.003095525755141715, 0.047673426743404586] [0.6994212265125732, -0.39871937375839495, 0.02229672622453182]


[ WARN] [1740651961.825618155, 651.109000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651961.958361257, 651.239000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651962.041873408, 651.321000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651962.110563476, 651.375000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651962.181730541, 651.437000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651962.321344180, 651.572000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651962.394589188, 651.644000000]: New joint state for joi

3 [3.141177646230598, -0.0027522792443199773, 0.054185056894009084] [0.6993540772779313, -0.29887599422475136, 0.02226546656411075]


[ WARN] [1740651971.666632887, 660.570000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651971.738887679, 660.641000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


[INFO] [1740651971.941707, 660.840000]: Move successful to position: [0.7393540772779313, -0.29887599422475136, 0.12226546656411075] and RPY: [3.141177646230598, -0.0027522792443199773, 0.054185056894009084]


[ WARN] [1740651971.869001145, 660.769000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.011055738, 660.904000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.165884803, 661.054000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.286854450, 661.172000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.360063233, 661.244000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.421876403, 661.305000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651972.632836997, 661.512000000]: New joint state for joi

In [29]:
# move the minipulator to the aim position
for T in aim_transforms:
    transform_matrix_x_180 = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])
    transform_matrix_z_90 = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    pick_rpy, pick_pos = matrix_to_rpy_and_translation(T@transform_matrix_x_180@transform_matrix_z_90)
    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.0, 0, 0.0])]
    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
    pick_place.move(pick_pos_, pick_rpy)

[ERROR] [1740651685.787344, 381.987000]: Target position z=0.0 violates minimum height constraint
[ WARN] [1740651686.170988182, 382.370000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


[INFO] [1740651688.098913, 384.240000]: Move successful to position: [0.3, 0.0, 0.04725] and RPY: [3.141592653589793, 0.0, -1.5707963267948966]
[INFO] [1740651690.323563, 386.446000]: Move successful to position: [0.3, 0.0, 0.0945] and RPY: [3.141592653589793, 0.0, -1.5707963267948966]


[ WARN] [1740651691.606641563, 387.702000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651691.945850810, 388.037000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651692.352395028, 388.437000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651692.826439113, 388.902000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651692.962533348, 389.036000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651700.486235343, 396.439000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651701.436867125, 397.372000000]: New joint state for joi

In [31]:
# pick and place the first cube 
transform_matrix_x_180 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])
transform_matrix_z_90 = np.array([
    [0, -1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

pick_rpy, pick_pos = matrix_to_rpy_and_translation(Ts[0])
pick_pos_ = [a + b for a, b in zip(pick_pos, [0.04, 0, 0.00])]
pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
    
place_rpy, place_pos = matrix_to_rpy_and_translation(aim_transforms[0]@transform_matrix_x_180@transform_matrix_z_90)
place_pos_ = [a + b for a, b in zip(place_pos, [0.0, 0, 0.03])]

pick_place.pick_and_place(
    pick_pos=pick_pos_,
    pick_rpy=pick_rpy,
    place_pos=place_pos_,
    place_rpy=place_rpy
)

[INFO] [1740651739.894298, 435.161000]: Restoring to initial joint values...


[ WARN] [1740651744.995353085, 440.172000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651745.063322660, 440.238000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651745.127941985, 440.302000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651745.539108245, 440.704000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651745.607009304, 440.770000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651745.811444252, 440.970000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651746.150316390, 441.303000000]: New joint state for joi

KeyboardInterrupt: 

[ WARN] [1740651758.773434225, 453.607000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651763.073469599, 457.803000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.


In [ ]:
# pick and place following cubes 
for i in range(1, Ts.__len__()-1):
    ptransform_matrix_x_180 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
    ])
    transform_matrix_z_90 = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    pick_rpy, pick_pos = matrix_to_rpy_and_translation(Ts[i])
    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.0, 0, 0.008])]
    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]

    place_rpy, place_pos = matrix_to_rpy_and_translation(aim_transforms[i]@transform_matrix_x_180@transform_matrix_z_90)
    place_pos_ = [a + b for a, b in zip(place_pos, [0.0, 0, 0.03])]

    pick_place.pick_and_place(
        pick_pos=pick_pos_,
        pick_rpy=pick_rpy,
        place_pos=place_pos_,
        place_rpy=place_rpy
    )

[ WARN] [1740651410.670710522, 116.037000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651411.359657367, 116.712000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651411.419757292, 116.771000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651411.487174363, 116.838000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651411.826000889, 117.171000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651412.028521247, 117.370000000]: New joint state for joint 'panda_joint1' is not newer than the previous state. Assuming your rosbag looped.
[ WARN] [1740651413.118725361, 118.440000000]: New joint state for joi